# Pré processamento de texto

O pré processamento dos textos é uma etapa fundamental para um modelo de Natural Language Processing (NLP), e consiste em transformar o texto original para uma forma mais adequada para que a máquina possa desempenhar melhor sobre ele.[1][2][3]

As etapas básicas, amplamente utilizadas, de um pré processamento são a tokenização, Lower casing, remoção de stop words, Stemming e Lematização.

A tokenização consiste em transformar a sentença em uma lista de palavras, assim separando umas das outras.

O lower casing é converter todas as letras de todas as palavras para minúsculas, para que palavras iguais, mas com diferença entre maiúscula (por terem sido utilizadas no começo de uma sentença e no fim de outra, por exemplo) sejam padronizadas para ser tratadas como iguais.

A rmoção de stopwords.

# Importação de Bibliotecas

In [1]:

import pandas as pd
from pre_processing2 import Pipeline

import re
import spacy

import nltk
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer


import gensim.corpora as corpora
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary



/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [2]:
### !scp /mnt/nas/trf1-dados

# Carregando os Dados

In [2]:

# newsgroup = fetch_20newsgroups(subset="train",remove=('headers', 'footers', 'quotes'))
# dados = pd.DataFrame(newsgroup.data)
# dados
dados = pd.read_parquet('/mnt/home/breno.beleza/datasets/stj-parquet.gzip')
dados = pd.DataFrame(dados['decisao'][:10000])
dados

decisao
0    EMENTA\nHABEAS CORPUS. CONDIÇÕES DO SURSIS. PR...
1    EMENTA\nRECURSO ORDINÁRIO CONSTITUCIONAL EM HA...
2    ACÓRDÃO\nVistos e relatados estes autos em que...
3    ACÓRDÃO\nVistos e relatados estes autos em que...
4    ACÓRDÃO\nVistos e relatados estes autos em que...
..                                                 ...
446  DECISÃO<br>Cuida-se deagravo interposto por JO...
447  DECISÃO<br>Cuida-se deagravo interposto por MU...
448  DECISÃO<br>Cuida-se deagravo interposto por MA...
449  DECISÃO<br>Cuida-se deagravo interposto por GI...
450  DECISÃO<br>Cuida-se deagravo interposto por MA...

[10000 rows x 1 columns]

In [3]:
dados = dados.reset_index(drop=True)

# Definindo um pré-processamento simples

### Remoção de caracteres não alfabéticos

In [4]:
n_alpha_removal = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in dados['decisao'])

### Função para limpar os dados
Lematizar e remover stop_words

Lematizar consiste em substituir as variantes de uma palavra pela original(palavras no plural, verbos flexionados, etc). Exemplo: substituir "amigos" e "amigas" por "amigo".

Stopwords são palavras comuns da língua com influências desprezíveis para NLP. Exemplos de stopwords do Português: "a", "ainda", "alguma", "uma", etc.


In [5]:
!python -m spacy download pt_core_news_sm
!python -m spacy download pt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 10.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.4.0/pt_core_news_sm-3.4.0-py3-none-any.whl (13.0 MB)

[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [6]:
nlp = spacy.load("pt_core_news_sm", disable=["ner", "parser"])

def cleaning(doc):
  txt=[token.lemma_ for token in doc if not token.is_stop]
  return txt

In [7]:
txt = [cleaning(doc) for doc in nlp.pipe(n_alpha_removal, batch_size=1000)] ## Deve demorar alguns minutos

In [8]:
dados_preprocessados_simples = pd.DataFrame({'clean':txt})
dados_preprocessados_simples

clean
0     [ementar, habeo, corpus, condi, es, sursi, pre...
1     [ementar, recurso, ordin, rio, constitucional,...
2     [ac, rd, vistos, relatar, auto, s, parte, acim...
3     [ac, rd, vistos, relatar, auto, s, parte, acim...
4     [ac, rd, vistos, relatar, auto, s, parte, acim...
...                                                 ...
9995  [decis, br, cuidar, deagravo, interposto, joce...
9996  [decis, br, cuidar, deagravo, interposto, muni...
9997  [decis, br, cuidar, deagravo, interposto, mari...
9998  [decis, br, cuidar, deagravo, interposto, Gilb...
9999  [decis, br, cuidar, deagravo, interposto, Marc...

[10000 rows x 1 columns]

#### Comparação entre antes e depois do pré-processamento simples para o primeiro texto do dataset

In [9]:
print(f'Texto antes do preprocessamento:\n{dados.decisao[0]}\n\n\nTexto depois do preprocessamento:\n {dados_preprocessados_simples.clean[0]}')

Texto antes do preprocessamento:
EMENTA
HABEAS CORPUS. CONDIÇÕES DO SURSIS. PRESTAÇÃO DE SERVIÇOS À COMUNIDADE SUBSTITUÍDA, PELO TRIBUNAL LOCAL, POR LIMITAÇÃO DE FINAL DE SEMANA, NO JULGAMENTO DE RECURSO DE APELAÇÃO INTERPOSTO EXCLUSIVAMENTE PELO SENTENCIADO. SITUAÇÃO, NA HIPÓTESE, MAIS GRAVOSA. PARTE DISPOSITIVA DO ÉDITO DE PRIMEIRO GRAU QUE, TODAVIA, PASSOU EM JULGADO PARA A ACUSAÇÃO, SEM QUE O MINISTÉRIO PÚBLICO REQUERESSE, NA FORMA E TEMPO ADEQUADOS, SUA REFORMA. PRECLUSÃO DO DIREITO DE BUSCAR QUAISQUER AGRAVAMENTOS DE PENA. CORREÇÃO DE OFÍCIO. IMPOSSIBILIDADE. PRINCÍPIO DA NE REFORMATIO IN PEJUS. PRECEDENTES. LIMINAR RATIFICADA. ORDEM DE HABEAS CORPUS CONCEDIDA.
1. Hipótese na qual o Juiz de primeiro grau na sentença fixou, como condições dosursis, 1) a prestação de 120 horas de serviços à comunidade; e 2) o comparecimento pessoal e obrigatório do Paciente em Juízo, bimestralmente, para informar e justificar suas atividades. Todavia, o Tribunala quo, em julgamento do recurso de ap

### Agora utilizando a pre_processing do AILab

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /mnt/home/breno.beleza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /mnt/home/breno.beleza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
stop_words = stopwords.words('portuguese')
# stop_words = stopwords.words('english')


In [13]:
first =['nacional','caso','lei','processo','direito','processual','civil','administrativo','tributário','agravo','instrumento','constitucional','penal','art','sentença','lei','mantida','nº']


stop_words = stop_words + first

def remove_stop_words(data_words,stop_words=stop_words):
    return [word for word in data_words if word not in stop_words]

In [14]:
def pre_process(dataframe,texto):
    dataframe['processed_text'] = dataframe[texto].apply(lambda x: Pipeline(disabled_steps=[ "transform_ner", "lemmatize" , "spellcheck", "stem", "transform_unknown", 'transform_syn'], embedding=False).apply(x, verbose=False))
    # dataframe['processed_text'] = dataframe[texto].apply(lambda x: Pipeline().apply(x, verbose=False))
    dataframe['processed_text'] = dataframe['processed_text'].apply(remove_stop_words)
    # dataframe = bigram_and_trigram(dataframe)    
    return dataframe.processed_text

In [15]:
dados_preprocessados_completo = pre_process(dados,'decisao') ## Deve demorar alguns minutos

/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1001: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn("Unknown precedent: '%s'", precedent)
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1001: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn("Unknown precedent: '%s'", precedent)
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1484: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1484: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  logging.warn(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pre_processing2/functions/transformation.py:1001: DeprecationWarning: The 'warn

In [16]:
dados_preprocessados_completo

0       [ementa, habeas, corpus, condicoes, sursis, pr...
1       [ementa, recurso, ordinario, habeas, corpus, h...
2       [acordao, vistos, relatados, autos, partes, ac...
3       [acordao, vistos, relatados, autos, partes, ac...
4       [acordao, vistos, relatados, autos, partes, ac...
                              ...                        
9995    [decisaobrcuidase, deagravo, interposto, jocem...
9996    [decisaobrcuidase, deagravo, interposto, munic...
9997    [decisaobrcuidase, deagravo, interposto, maric...
9998    [decisaobrcuidase, deagravo, interposto, gilbe...
9999    [decisaobrcuidase, deagravo, interposto, marce...
Name: processed_text, Length: 10000, dtype: object

#### Comparação entre o antes e depois do pré-processamento do AILab para o primeiro texto do dataset

In [17]:
print(f'Texto antes do preprocessamento:\n{dados.decisao[0]}\n\n\nTexto depois do preprocessamento:\n {dados_preprocessados_completo[0]}')

Texto antes do preprocessamento:
EMENTA
HABEAS CORPUS. CONDIÇÕES DO SURSIS. PRESTAÇÃO DE SERVIÇOS À COMUNIDADE SUBSTITUÍDA, PELO TRIBUNAL LOCAL, POR LIMITAÇÃO DE FINAL DE SEMANA, NO JULGAMENTO DE RECURSO DE APELAÇÃO INTERPOSTO EXCLUSIVAMENTE PELO SENTENCIADO. SITUAÇÃO, NA HIPÓTESE, MAIS GRAVOSA. PARTE DISPOSITIVA DO ÉDITO DE PRIMEIRO GRAU QUE, TODAVIA, PASSOU EM JULGADO PARA A ACUSAÇÃO, SEM QUE O MINISTÉRIO PÚBLICO REQUERESSE, NA FORMA E TEMPO ADEQUADOS, SUA REFORMA. PRECLUSÃO DO DIREITO DE BUSCAR QUAISQUER AGRAVAMENTOS DE PENA. CORREÇÃO DE OFÍCIO. IMPOSSIBILIDADE. PRINCÍPIO DA NE REFORMATIO IN PEJUS. PRECEDENTES. LIMINAR RATIFICADA. ORDEM DE HABEAS CORPUS CONCEDIDA.
1. Hipótese na qual o Juiz de primeiro grau na sentença fixou, como condições dosursis, 1) a prestação de 120 horas de serviços à comunidade; e 2) o comparecimento pessoal e obrigatório do Paciente em Juízo, bimestralmente, para informar e justificar suas atividades. Todavia, o Tribunala quo, em julgamento do recurso de ap

## Modelagem de tópicos para cada caso

In [18]:
# tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words,
#                                    max_df=0.5,
#                                    min_df=0.0)

def generate_lda(data_ready, number_of_topics ):
    
    # data_ready = pre_process(dataframe)
    id2word = corpora.Dictionary(data_ready)
    temp = id2word[0]
    corpus = [id2word.doc2bow(text) for text in data_ready]
    lda_model = LdaModel(corpus=corpus,
                         id2word=id2word,
                         num_topics=number_of_topics,
                         random_state=42,
                         update_every=1,
                         chunksize=len(corpus),
                         passes=10,
                         alpha='auto',
                         iterations=100)
    return lda_model, corpus, id2word


#### Primeiro a modelagem sem pré-processamento

In [19]:
# lda = (n_components=20)
# lda.fit(tfidf_vectorizer.fit_transform(dados[0]))
dados_naoprocessados = dados['decisao'].apply(lambda x: x.split(" "))
dados_naoprocessados


0       [EMENTA\nHABEAS, CORPUS., CONDIÇÕES, DO, SURSI...
1       [EMENTA\nRECURSO, ORDINÁRIO, CONSTITUCIONAL, E...
2       [ACÓRDÃO\nVistos, e, relatados, estes, autos, ...
3       [ACÓRDÃO\nVistos, e, relatados, estes, autos, ...
4       [ACÓRDÃO\nVistos, e, relatados, estes, autos, ...
                              ...                        
9995    [DECISÃO<br>Cuida-se, deagravo, interposto, po...
9996    [DECISÃO<br>Cuida-se, deagravo, interposto, po...
9997    [DECISÃO<br>Cuida-se, deagravo, interposto, po...
9998    [DECISÃO<br>Cuida-se, deagravo, interposto, po...
9999    [DECISÃO<br>Cuida-se, deagravo, interposto, po...
Name: decisao, Length: 10000, dtype: object

In [20]:
lda_model1, corpus1, id2word1 = generate_lda(dados_naoprocessados, number_of_topics=7)

In [21]:
# pyLDAvis.sklearn.prepare(lda, tfidf_vectorizer.fit_transform(dados[0]), tfidf_vectorizer)
viz_data1 = pyLDAvis.gensim_models.prepare(lda_model1, corpus1, id2word1)
pyLDAvis.display(viz_data1)

/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


#### Agora com o pré-processamento simples

In [22]:
# dados_preprocessados_simples['clean'] = dados_preprocessados_simples['clean'].apply(lambda x: " ".join(x))
dados_preprocessados_simples
lda_model2, corpus2, id2word2 = generate_lda(dados_preprocessados_simples.clean, number_of_topics=7)

In [23]:
# lda2 = LatentDirichletAllocation(n_components=20)
# lda2.fit(tfidf_vectorizer.fit_transform(dados_preprocessados_simples['clean']))

viz_data2 = pyLDAvis.gensim_models.prepare(lda_model2, corpus2, id2word2)
pyLDAvis.display(viz_data2)

/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [24]:
# pyLDAvis.sklearn.prepare(lda2, tfidf_vectorizer.fit_transform(dados_preprocessados_simples['clean']), tfidf_vectorizer)

#### Por último com o pré-processamento do AILab

In [25]:
# dados_preprocessados_completo = dados_preprocessados_completo.apply(lambda x: " ".join(x))
dados_preprocessados_completo

0       [ementa, habeas, corpus, condicoes, sursis, pr...
1       [ementa, recurso, ordinario, habeas, corpus, h...
2       [acordao, vistos, relatados, autos, partes, ac...
3       [acordao, vistos, relatados, autos, partes, ac...
4       [acordao, vistos, relatados, autos, partes, ac...
                              ...                        
9995    [decisaobrcuidase, deagravo, interposto, jocem...
9996    [decisaobrcuidase, deagravo, interposto, munic...
9997    [decisaobrcuidase, deagravo, interposto, maric...
9998    [decisaobrcuidase, deagravo, interposto, gilbe...
9999    [decisaobrcuidase, deagravo, interposto, marce...
Name: processed_text, Length: 10000, dtype: object

In [26]:
lda_model3, corpus3, id2word3 = generate_lda(dados_preprocessados_completo, number_of_topics=7)

In [27]:
# lda3 = LatentDirichletAllocation(n_components=20)
# lda3.fit(tfidf_vectorizer.fit_transform(dados_preprocessados_completo))

viz_data3 = pyLDAvis.gensim_models.prepare(lda_model3, corpus3, id2word3)
pyLDAvis.display(viz_data3)


/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/mnt/home/breno.beleza/ENV/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [28]:
# pyLDAvis.sklearn.prepare(lda3, tfidf_vectorizer.fit_transform(dados_preprocessados_completo), tfidf_vectorizer)

# Bibliografia

[1] - [towardsdatascience.com/pre-processing-in-natural-language-machine-learning](https://towardsdatascience.com/pre-processing-in-natural-language-machine-learning-898a84b8bd47)

[2] - [towardsdatascience.com/text-preprocessing-in-natural-language-processing-using-python](https://towardsdatascience.com/text-preprocessing-in-natural-language-processing-using-python-6113ff5decd8#:~:text=Significance%20of%20text%20preprocessing%20in,process%20of%20building%20a%20model.)

[3] - [towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template](https://towardsdatascience.com/nlp-text-preprocessing-a-practical-guide-and-template-d80874676e79)

